In [4]:
import pandas as pd
import numpy as np
import networkx as nx
import ast
import matplotlib.pyplot as plt
from datetime import datetime as dt
from random import randint
from collections import Counter
import math

## Get data

In [5]:
users = pd.read_csv('2018_02_20_Users.csv', encoding = "ISO-8859-1")

In [6]:
articles = pd.read_csv('2018_02_20_Articles.csv', encoding = "ISO-8859-1")

In [7]:
articles.head(n=5)

,userHash (S),articleURL (S),articleClicked (BOOL),articleClickedTime (S),articleComments (L),articlePopularity (L),articleSeenTime (L),articleSharedTime (L),articleTitle (S),sharerHash (L)
0,-318129462,http://bit.ly/Timoth%C3%A9e-Armie-YOU,True,"Sun, 11 Feb 2018 02:44:44 GMT",[],[],"[ { ""S"" : ""Sun, 11 Feb 2018 02:44:44 GMT"" }]",[],NaN,[]
1,-318129462,http://eonli.ne/2BZ0vBf,True,"Fri, 16 Feb 2018 17:28:00 GMT",[],[],"[ { ""S"" : ""Fri, 16 Feb 2018 17:28:00 GMT"" }]",[],NaN,[]
2,-318129462,http://www.eonline.com/news/911768/jamie-dorna...,True,"Fri, 16 Feb 2018 17:28:08 GMT",[],[],"[ { ""S"" : ""Fri, 16 Feb 2018 17:28:08 GMT"" }]",[],NaN,[]
3,-318129462,https://offbeat.topix.net/slideshow/18943,True,"Sun, 11 Feb 2018 02:52:14 GMT",[],[],"[ { ""S"" : ""Sun, 11 Feb 2018 02:52:14 GMT"" }]",[],NaN,[]
4,-318129462,https://rewire.news/article/2018/02/14/south-a...,True,"Fri, 16 Feb 2018 17:18:31 GMT",[],[],"[ { ""S"" : ""Fri, 16 Feb 2018 17:18:29 GMT"" }]",[],NaN,[]


## Functions

In [8]:
def makeGraph(timestamp):
    timestamp = dt.strptime(timestamp, "%a, %d %b %Y %H:%M:%S %Z")
    beforetime_DF = pd.DataFrame()
    for i in range(0, len(forgraph)):
        temp_date_list = ast.literal_eval(forgraph.iloc[[i]]["articleSeenTime (L)"][i])
        for k in range(0, len(temp_date_list)):
            temp_date = temp_date_list[k].get("S")
            date_obj = dt.strptime(temp_date, "%a, %d %b %Y %H:%M:%S %Z")
            if date_obj <= timestamp:
                beforetime_DF = beforetime_DF.append(forgraph.iloc[[i]], ignore_index=True)
                break
                
    sharerHash_list = beforetime_DF['sharerHash (L)']
    userHash_list = beforetime_DF["userHash (S)"]
    
    edges_list = []
    for i in range(0, len(sharerHash_list)):
        temp = ast.literal_eval(sharerHash_list[i])
        for k in range(0, len(temp)):
            edges_list.append((str(userHash_list[i]), temp[k].get("S")))
            
    weighted_edges = []
    i = 0
    while i < len(edges_list)-2:
        weight = 1
        while ((edges_list[i][0] == edges_list[i+1][0]) and (edges_list[i][1] == edges_list[i+1][1])):
            weight = weight + 1
            i = i + 1
        weighted_edges.append((edges_list[i][0], edges_list[i][1], weight))
        i = i + 1
        
    interactions = nx.DiGraph()
    interactions.add_weighted_edges_from(weighted_edges)
    return interactions

In [20]:
def getRandomInteraction(all_interactions):
    index = randint(0, len(all_interactions)-1)
    currUserHash = all_interactions.iloc[[index]]['userHash (S)'][index]
    currSharerHash_list = ast.literal_eval(all_interactions.iloc[[index]]['sharerHash (L)'][index])
    currSeenTime_list = ast.literal_eval(all_interactions.iloc[[index]]['articleSeenTime (L)'][index])
    if len(currSharerHash_list) == 1:
        currSharerHash = currSharerHash_list[0].get("S")
        currSeenTime = currSeenTime_list[0].get("S")
    else:
        t_index = randint(0, len(currSharerHash_list)-1)
        currSharerHash = currSharerHash_list[t_index].get("S")
        currSeenTime= currSeenTime_list[t_index].get("S")
    return [str(currUserHash), str(currSharerHash), str(currSeenTime)]

In [21]:
def weightedCommonNeighbors(x, y, currGraph):
    x_neighbors = currGraph.neighbors(x)
    y_neighbors = currGraph.neighbors(y)
    x_y_intersection = list(set(x_neighbors) & set(y_neighbors))
    score = 0
    for n in x_y_intersection:
        x_n_weight = currGraph.edge[x][n]['weight']
        y_n_weight = currGraph.edge[y][n]['weight']
        score = score + (x_n_weight + y_n_weight)/2
    return score

In [63]:
def weightedAdamicAdar(x, y, currGraph):
    x_neighbors = currGraph.neighbors(x)
    y_neighbors = currGraph.neighbors(y)
    x_y_intersection = list(set(x_neighbors) & set(y_neighbors))
    score = 0
    for z in x_y_intersection:
        x_z_weight = currGraph.edge[x][z]['weight']
        y_z_weight = currGraph.edge[y][z]['weight']
        z_neighbors = currGraph.neighbors(z)
        z_score = 0
        for z_prime in z_neighbors:
            z_z_prime_weight = currGraph.edge[z][z_prime]['weight']
            z_score = z_score + z_z_prime_weight
        if z_score == 0:
            score = score + ((x_z_weight + y_z_weight)/2)
        else:
            score = score + (((x_z_weight + y_z_weight)/2) * (1/math.log10(z_score)))
    return score

In [68]:
def weightedPrefAttachment(x, y, currGraph):
    x_neighbors = currGraph.neighbors(x)
    y_neighbors = currGraph.neighbors(y)
    x_sum = 0
    for x_prime in x_neighbors:
        x_sum = x_sum + currGraph.edge[x][x_prime]['weight']
    y_sum = 0
    for y_prime in y_neighbors:
        y_sum = y_sum + currGraph.edge[y][y_prime]['weight']
    score = x_sum * y_sum
    return score

## Run

In [13]:
forgraph = articles[articles['sharerHash (L)'] != '[]']
forgraph = forgraph.reset_index()

### Method 0: Base method

In [14]:
to_pred = getRandomInteraction(forgraph)
print(to_pred)

['-1717522715', '140759912', 'Mon, 19 Feb 2018 15:43:56 GMT']


In [15]:
currGraph = makeGraph(to_pred[2])
#nx.draw(currGraph)
#plt.show()

### Method 1: Sample only from those nodes who have previously had interactions

In [53]:
curr = getRandomInteraction(forgraph)
g = makeGraph(curr[2])
while len(g[curr[1]]) == 0:
    curr = getRandomInteraction(forgraph)
    g = makeGraph(curr[2])

In [69]:
weightedCommonNeighbors(curr[0], curr[1], g)

2.0

In [70]:
weightedAdamicAdar(curr[0], curr[1], g)

2.0

In [71]:
weightedPrefAttachment(curr[0], curr[1], g)

86

### Method 2: Assign all nodes a base neighbors weight

In [45]:
b_weight = 1



## Weighted Common Neighbors Scratch

In [133]:
len(currGraph.neighbors(to_pred[0]))

36

In [134]:
len(currGraph.neighbors(to_pred[1]))

0

In [16]:
len(currGraph[to_pred[1]])

0

In [127]:
currGraph[to_pred[0]]

{'-1540964113': {'weight': 1},
 '-202640102': {'weight': 5},
 '-528531470': {'weight': 1},
 '-542803728': {'weight': 2},
 '-738178768': {'weight': 2},
 '-809509582': {'weight': 1},
 '-866615615': {'weight': 1},
 '1614547276': {'weight': 2},
 '1801204512': {'weight': 1},
 '1802160644': {'weight': 1},
 '1837954089': {'weight': 1},
 '2129326999': {'weight': 1},
 '2129681376': {'weight': 2},
 '2407815': {'weight': 1},
 '51359063': {'weight': 1},
 '546618692': {'weight': 1}}

In [138]:
sharerHashes = forgraph["sharerHash (L)"]
sharerHashOcc = []
for i in range(0, len(sharerHashes)):
    temp = ast.literal_eval(sharerHashes[i])
    for k in range(0, len(temp)):
        sharerHashOcc.append(temp[k].get("S"))
            
Counter(sharerHashOcc)

Counter({'-1004379730': 1,
         '-1014642500': 1,
         '-1017429739': 2,
         '-1020376266': 1,
         '-1049645274': 5,
         '-1057059289': 1,
         '-1063750480': 1,
         '-1069486869': 1,
         '-1072741861': 1,
         '-1080635277': 1,
         '-1081875553': 1,
         '-108587042': 1,
         '-1102884016': 8,
         '-1111353745': 1,
         '-1113822672': 1,
         '-1115968853': 1,
         '-1137319660': 1,
         '-1140639860': 2,
         '-1141535170': 2,
         '-1141622016': 1,
         '-1147885674': 1,
         '-1163705852': 1,
         '-1174757635': 1,
         '-1175428812': 1,
         '-1178880727': 1,
         '-1183089852': 1,
         '-1189135403': 1,
         '-1190640033': 1,
         '-1208985214': 1,
         '-1213378920': 1,
         '-1220673407': 1,
         '-1224626268': 1,
         '-1228697902': 6,
         '-1229375078': 4,
         '-1236559331': 1,
         '-1241988937': 1,
         '-1263565376': 1,
  

## Make Graph Scratch

In [9]:
forgraph = articles[articles['sharerHash (L)'] != '[]']

In [13]:
len(forgraph)

1442

In [16]:
forgraph.head(n=5)

,userHash (S),articleURL (S),articleClicked (BOOL),articleClickedTime (S),articleComments (L),articlePopularity (L),articleSeenTime (L),articleSharedTime (L),articleTitle (S),sharerHash (L)
14,-1949465643,http://jstreet.org/press-releases/j-street-opp...,False,NaN,"[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""10"" }]","[ { ""S"" : ""Tue, 13 Feb 2018 01:21:46 GMT"" }]","[ { ""S"" : ""02/12/2018 6:54pm"" }]",J Street Opposes Confirmation of Kenneth Marcu...,"[ { ""S"" : ""-1174757635"" }]"
19,-1748993256,http://hill.cm/EOrPLMZ,False,NaN,"[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""3 "" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:06:00 GMT"" }]","[ { ""S"" : ""NULL"" }]",White House refused to release photo of Trump ...,"[ { ""S"" : ""2407815"" }]"
20,-1748993256,http://hill.cm/X9HpS96,False,NaN,"[ { ""S"" : ""BREAKING: FBI officials were alert...","[ { ""S"" : ""1"" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:04:49 GMT"" }]","[ { ""S"" : ""02/15/2018 8:05am"" }]",FBI was warned last year of school shooting th...,"[ { ""S"" : ""1101628624"" }]"
21,-1748993256,http://stanford.io/2ClZ68d,True,"Thu, 15 Feb 2018 14:32:01 GMT","[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""44"" }]","[ { ""S"" : ""Thu, 15 Feb 2018 14:32:01 GMT"" }]","[ { ""S"" : ""02/15/2018 8:45am"" }]","http://stanford.io/2ClZ68d</a> <br> <a class=""...","[ { ""S"" : ""46193696"" }]"
22,-1748993256,http://t.ted.com/gmdx0iv,False,NaN,"[ { ""S"" : ""Practical money advice from a self...","[ { ""S"" : ""4"" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:04:49 GMT"" }]","[ { ""S"" : ""02/16/2018 10:00am"" }]",6 simple tips for investment beginners,"[ { ""S"" : ""82931"" }]"


In [30]:
forgraph = forgraph.reset_index()

In [145]:
forgraph.head(n=50)

,index,userHash (S),articleURL (S),articleClicked (BOOL),articleClickedTime (S),articleComments (L),articlePopularity (L),articleSeenTime (L),articleSharedTime (L),articleTitle (S),sharerHash (L)
0,14,-1949465643,http://jstreet.org/press-releases/j-street-opp...,False,NaN,"[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""10"" }]","[ { ""S"" : ""Tue, 13 Feb 2018 01:21:46 GMT"" }]","[ { ""S"" : ""02/12/2018 6:54pm"" }]",J Street Opposes Confirmation of Kenneth Marcu...,"[ { ""S"" : ""-1174757635"" }]"
1,19,-1748993256,http://hill.cm/EOrPLMZ,False,NaN,"[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""3 "" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:06:00 GMT"" }]","[ { ""S"" : ""NULL"" }]",White House refused to release photo of Trump ...,"[ { ""S"" : ""2407815"" }]"
2,20,-1748993256,http://hill.cm/X9HpS96,False,NaN,"[ { ""S"" : ""BREAKING: FBI officials were alert...","[ { ""S"" : ""1"" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:04:49 GMT"" }]","[ { ""S"" : ""02/15/2018 8:05am"" }]",FBI was warned last year of school shooting th...,"[ { ""S"" : ""1101628624"" }]"
3,21,-1748993256,http://stanford.io/2ClZ68d,True,"Thu, 15 Feb 2018 14:32:01 GMT","[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""44"" }]","[ { ""S"" : ""Thu, 15 Feb 2018 14:32:01 GMT"" }]","[ { ""S"" : ""02/15/2018 8:45am"" }]","http://stanford.io/2ClZ68d</a> <br> <a class=""...","[ { ""S"" : ""46193696"" }]"
4,22,-1748993256,http://t.ted.com/gmdx0iv,False,NaN,"[ { ""S"" : ""Practical money advice from a self...","[ { ""S"" : ""4"" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:04:49 GMT"" }]","[ { ""S"" : ""02/16/2018 10:00am"" }]",6 simple tips for investment beginners,"[ { ""S"" : ""82931"" }]"
5,23,-1748993256,https://collegepulse.com/survey/5a85e6d386f8d3...,False,NaN,"[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""1"" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:04:49 GMT"" }]","[ { ""S"" : ""02/16/2018 9:40am"" }]",www.collegepulse.com</a>,"[ { ""S"" : ""1236633202"" }]"
6,24,-1748993256,https://gearjunkie.com/sasha-digiulian-ice-cli...,False,NaN,"[ { ""S"" : ""I spoke with <a class=\""profileLin...","[ { ""S"" : ""109"" }]","[ { ""S"" : ""Tue, 20 Feb 2018 18:12:37 GMT"" }]","[ { ""S"" : ""02/20/2018 12:08pm"" }]",First-Time Frozen: Sasha DiGiulian Swings Axes...,"[ { ""S"" : ""1396533814"" }]"
7,31,-1748993256,https://www.google.com/amp/s/www.vox.com/platf...,False,NaN,"[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""4"" }]","[ { ""S"" : ""Thu, 15 Feb 2018 14:33:14 GMT"" }]","[ { ""S"" : ""02/15/2018 8:58am"" }]","<div class=""_6l- __c_""><div class=""uiScaledIma...","[ { ""S"" : ""1497619900"" }]"
8,32,-1748993256,https://www.nytimes.com/2017/11/15/magazine/my...,False,NaN,"[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""45"" }]","[ { ""S"" : ""Tue, 20 Feb 2018 18:00:47 GMT"" }]","[ { ""S"" : ""feed_story_id_5a8c603f65aa07d43202...",My Wife Is Done With Sex. Can I Turn Elsewhere?,"[ { ""S"" : ""1802160644"" }]"
9,33,-1748993256,https://www.princeton.edu/news/2018/02/12/arti...,True,"Fri, 16 Feb 2018 15:14:36 GMT","[ { ""S"" : ""Artist Walter Hood has been commis...","[ { ""S"" : ""2"" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:14:31 GMT"" }]","[ { ""S"" : ""02/16/2018 10:11am"" }]",Artist Walter Hood commissioned for Scudder Pl...,"[ { ""S"" : ""546618692"" }]"


In [37]:
cleanSharerHash = forgraph['sharerHash (L)']

In [55]:
userHash_list = articles["userHash (S)"]

In [83]:
edges_list = []
for i in range(0, len(cleanSharerHash)):
    temp = ast.literal_eval(cleanSharerHash[i])
    for k in range(0, len(temp)):
        edges_list.append((userHash_list[i], temp[k].get("S"), 1))

In [84]:
edges_list

[(-318129462, '-1174757635', 1),
 (-318129462, '2407815', 1),
 (-318129462, '1101628624', 1),
 (-318129462, '46193696', 1),
 (-318129462, '82931', 1),
 (-318129462, '1236633202', 1),
 (-318129462, '1396533814', 1),
 (-318129462, '1497619900', 1),
 (635926102, '1802160644', 1),
 (635926102, '546618692', 1),
 (635926102, '2407815', 1),
 (635926102, '1689492990', 1),
 (635926102, '455884322', 1),
 (635926102, '455884322', 1),
 (-1949465643, '455884322', 1),
 (-1949465643, '455884322', 1),
 (-1949465643, '455884322', 1),
 (-1949465643, '455884322', 1),
 (-1949465643, '455884322', 1),
 (-1949465643, '455884322', 1),
 (-1748993256, '621378011', 1),
 (-1748993256, '621378011', 1),
 (-1748993256, '2129681376', 1),
 (-1748993256, '2129681376', 1),
 (-1748993256, '-460797268', 1),
 (-1748993256, '2111871096', 1),
 (-1748993256, '-1049645274', 1),
 (-1748993256, '-1163705852', 1),
 (-1748993256, '-173132451', 1),
 (-1748993256, '761826859', 1),
 (-1748993256, '431086251', 1),
 (-1748993256, '4310

In [81]:
edges_list[0][0] == edges_list[1][0]

True

In [95]:
weighted_edges = []
i = 0
while i < len(edges_list)-1:
    weight = 1
    while ((edges_list[i][0] == edges_list[i+1][0]) and (edges_list[i][1] == edges_list[i+1][1])):
        weight = weight + 1
        i = i + 1
    weighted_edges.append((edges_list[i][0], edges_list[i][1], weight))
    i = i + 1

In [96]:
weighted_edges

[(-318129462, '-1174757635', 1),
 (-318129462, '2407815', 1),
 (-318129462, '1101628624', 1),
 (-318129462, '46193696', 1),
 (-318129462, '82931', 1),
 (-318129462, '1236633202', 1),
 (-318129462, '1396533814', 1),
 (-318129462, '1497619900', 1),
 (635926102, '1802160644', 1),
 (635926102, '546618692', 1),
 (635926102, '2407815', 1),
 (635926102, '1689492990', 1),
 (635926102, '455884322', 2),
 (-1949465643, '455884322', 6),
 (-1748993256, '621378011', 2),
 (-1748993256, '2129681376', 2),
 (-1748993256, '-460797268', 1),
 (-1748993256, '2111871096', 1),
 (-1748993256, '-1049645274', 1),
 (-1748993256, '-1163705852', 1),
 (-1748993256, '-173132451', 1),
 (-1748993256, '761826859', 1),
 (-1748993256, '431086251', 4),
 (-1748993256, '443533532', 1),
 (-1748993256, '2575053', 1),
 (-1318276747, '2575053', 1),
 (-1318276747, '1294427060', 1),
 (-1318276747, '-1796215358', 1),
 (-1318276747, '65960851', 1),
 (-1318276747, '2407815', 1),
 (-1318276747, '1827199790', 1),
 (-1318276747, '156864

In [98]:
interactions =nx.DiGraph()
interactions.add_weighted_edges_from(weighted_edges)

In [99]:
interactions.out_degree(-1949465643, weight='weight')

6

In [100]:
nx.draw(interactions)
plt.show()

In [156]:
def makeGraph(timestamp):
    timestamp = dt.strptime(timestamp, "%a, %d %b %Y %H:%M:%S %Z")
    beforetime_DF = pd.DataFrame()
    for i in range(0, len(forgraph)):
        temp_date_list = ast.literal_eval(forgraph.iloc[[i]]["articleSeenTime (L)"][i])
        for k in range(0, len(temp_date_list)):
            temp_date = temp_date_list[k].get("S")
            date_obj = dt.strptime(temp_date, "%a, %d %b %Y %H:%M:%S %Z")
            if date_obj <= timestamp:
                beforetime_DF = beforetime_DF.append(forgraph.iloc[[i]], ignore_index=True)
                break
                
    sharerHash_list = beforetime_DF['sharerHash (L)']
    userHash_list = beforetime_DF["userHash (S)"]
    
    edges_list = []
    for i in range(0, len(sharerHash_list)):
        temp = ast.literal_eval(sharerHash_list[i])
        for k in range(0, len(temp)):
            edges_list.append((userHash_list[i], temp[k].get("S")))
            
    weighted_edges = []
    i = 0
    while i < len(edges_list)-1:
        weight = 1
        while ((edges_list[i][0] == edges_list[i+1][0]) and (edges_list[i][1] == edges_list[i+1][1])):
            weight = weight + 1
            i = i + 1
        weighted_edges.append((edges_list[i][0], edges_list[i][1], weight))
        i = i + 1
        
    interactions = nx.DiGraph()
    interactions.add_weighted_edges_from(weighted_edges)
    return interactions

In [126]:
temp_date = ast.literal_eval(articles["articleSeenTime (L)"][0])[0].get("S")

date_obj = dt.strptime(temp_date, "%a, %d %b %Y %H:%M:%S %Z")

In [154]:
timestamp = dt.strptime("Sun, 18 Feb 2018 23:10:40 GMT", "%a, %d %b %Y %H:%M:%S %Z")
beforetime_DF = pd.DataFrame()
for i in range(0, len(forgraph)):
    temp_date_list = ast.literal_eval(forgraph.iloc[[i]]["articleSeenTime (L)"][i])
    for k in range(0, len(temp_date_list)):
        temp_date = temp_date_list[k].get("S")
        date_obj = dt.strptime(temp_date, "%a, %d %b %Y %H:%M:%S %Z")
        if date_obj <= timestamp:
            beforetime_DF = beforetime_DF.append(forgraph.iloc[[i]], ignore_index=True)
            break

In [155]:
beforetime_DF

,index,userHash (S),articleURL (S),articleClicked (BOOL),articleClickedTime (S),articleComments (L),articlePopularity (L),articleSeenTime (L),articleSharedTime (L),articleTitle (S),sharerHash (L)
0,14,-1949465643,http://jstreet.org/press-releases/j-street-opp...,False,NaN,"[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""10"" }]","[ { ""S"" : ""Tue, 13 Feb 2018 01:21:46 GMT"" }]","[ { ""S"" : ""02/12/2018 6:54pm"" }]",J Street Opposes Confirmation of Kenneth Marcu...,"[ { ""S"" : ""-1174757635"" }]"
1,19,-1748993256,http://hill.cm/EOrPLMZ,False,NaN,"[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""3 "" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:06:00 GMT"" }]","[ { ""S"" : ""NULL"" }]",White House refused to release photo of Trump ...,"[ { ""S"" : ""2407815"" }]"
2,20,-1748993256,http://hill.cm/X9HpS96,False,NaN,"[ { ""S"" : ""BREAKING: FBI officials were alert...","[ { ""S"" : ""1"" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:04:49 GMT"" }]","[ { ""S"" : ""02/15/2018 8:05am"" }]",FBI was warned last year of school shooting th...,"[ { ""S"" : ""1101628624"" }]"
3,21,-1748993256,http://stanford.io/2ClZ68d,True,"Thu, 15 Feb 2018 14:32:01 GMT","[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""44"" }]","[ { ""S"" : ""Thu, 15 Feb 2018 14:32:01 GMT"" }]","[ { ""S"" : ""02/15/2018 8:45am"" }]","http://stanford.io/2ClZ68d</a> <br> <a class=""...","[ { ""S"" : ""46193696"" }]"
4,22,-1748993256,http://t.ted.com/gmdx0iv,False,NaN,"[ { ""S"" : ""Practical money advice from a self...","[ { ""S"" : ""4"" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:04:49 GMT"" }]","[ { ""S"" : ""02/16/2018 10:00am"" }]",6 simple tips for investment beginners,"[ { ""S"" : ""82931"" }]"
5,23,-1748993256,https://collegepulse.com/survey/5a85e6d386f8d3...,False,NaN,"[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""1"" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:04:49 GMT"" }]","[ { ""S"" : ""02/16/2018 9:40am"" }]",www.collegepulse.com</a>,"[ { ""S"" : ""1236633202"" }]"
6,31,-1748993256,https://www.google.com/amp/s/www.vox.com/platf...,False,NaN,"[ { ""S"" : ""NULL"" }]","[ { ""S"" : ""4"" }]","[ { ""S"" : ""Thu, 15 Feb 2018 14:33:14 GMT"" }]","[ { ""S"" : ""02/15/2018 8:58am"" }]","<div class=""_6l- __c_""><div class=""uiScaledIma...","[ { ""S"" : ""1497619900"" }]"
7,33,-1748993256,https://www.princeton.edu/news/2018/02/12/arti...,True,"Fri, 16 Feb 2018 15:14:36 GMT","[ { ""S"" : ""Artist Walter Hood has been commis...","[ { ""S"" : ""2"" }]","[ { ""S"" : ""Fri, 16 Feb 2018 15:14:31 GMT"" }]","[ { ""S"" : ""02/16/2018 10:11am"" }]",Artist Walter Hood commissioned for Scudder Pl...,"[ { ""S"" : ""546618692"" }]"
8,38,-1318276747,http://en.quizzapps.com/how-would-you-look-opp...,True,"Thu, 15 Feb 2018 00:31:49 GMT","[ { ""S"" : ""who is she??"" }]","[ { ""S"" : ""49"" }]","[ { ""S"" : ""Thu, 15 Feb 2018 00:31:47 GMT"" }]","[ { ""S"" : ""02/14/2018 12:09pm"" }]",Find Out How You'd Look Like As The Opposite G...,"[ { ""S"" : ""1689492990"" }]"
9,40,-1318276747,http://www.iflscience.com/brain/previously-und...,False,NaN,"[ { ""S"" : ""It's a fascinating language. There...","[ { ""S"" : ""642"" }]","[ { ""S"" : ""Fri, 09 Feb 2018 02:45:01 GMT"" }]","[ { ""S"" : ""02/08/2018 9:22pm"" }]",New Language Unknown To Science Discovered In ...,"[ { ""S"" : ""455884322"" }]"


In [1]:
currGraph = makeGraph("Sun, 18 Feb 2018 23:10:40 GMT")

NameError: name 'makeGraph' is not defined